## Importación de módulos

In [23]:
from collections import defaultdict
import cv2
import numpy as np
from ultralytics import YOLO
from shapely.geometry import LineString, Point

## Aplicación interactiva

In [25]:

model = YOLO('yolov8customv2.pt',task="detect")

video_path = "videos/traffic.mp4"
cap = cv2.VideoCapture(video_path)

track_history = defaultdict(lambda: [])
line_points = [(1200, 465), (100, 465)]  # line or region points

classes_to_count = [0,1,2,3,4,5,6,7,8,9,10,11]  # cars, motorcycles, bus, trucks

counting_line = LineString(line_points)

y1 = 420
y2 = 510

south_counts = 0
north_counts = 0
counting_list = []


while cap.isOpened():

    success, frame = cap.read()

    if success:
        
        results = model.track(frame, persist=True,classes=classes_to_count,show=False)

        half_width = frame.shape[1] // 2

        line_points_left1 = [(0, y1), (half_width, y1)]  # Izquierda 1, color: rojo
        line_points_left2 = [(half_width, y1), (frame.shape[1], y1)]  # Izquierda 2, color: azul
        line_points_right1 = [(0, y2), (half_width, y2)]  # Derecha 1, color: verde
        line_points_right2 = [(half_width, y2), (frame.shape[1], y2)]  # Derecha 2, color: amarillo

        # Get the boxes and track IDs
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        frame = results[0].plot()

        # Plot the tracks and calculate speed
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y))) 
            if len(track) > 10: 
                track.pop(0)

            # Calculate speed and infer direction based on the shape of the vehicle contour and tracking direction
            if len(track) >= 2:
                current_pos = np.array(track[-1])
                previous_pos = np.array(track[-2])
                speed = np.linalg.norm(current_pos - previous_pos)

                # Infer direction based on the shape of the vehicle contour and tracking direction
                contour = np.array([[int(x), int(y)], [int(x + w), int(y)], [int(x + w), int(y + h)], [int(x), int(y + h)]])
                rect = cv2.minAreaRect(contour)
                angle = rect[2]

                direction = "Norte" if current_pos[1] < previous_pos[1] else "Sur"

                distance = Point(track[-1]).distance(counting_line)
                if distance < 15:
                    if track_id not in counting_list:
                        counting_list.append(track_id)
                        if box[0] < counting_line.centroid.x:
                            north_counts += 1
                        else:
                            south_counts += 1

                south_text = 'Vehiculos Direccion Sur : ' + f'{south_counts}'
                north_text = 'Vehiculos Direccion Norte : ' + f'{north_counts}'

                #Ponemos contadores
                cv2.putText(frame, south_text, (950,50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                cv2.putText(frame, north_text, (100,50),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
                
                # Display speed and direction on the frame
                cv2.putText(frame, f"Speed: {speed:.2f} px/frame", (int(x), int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                cv2.putText(frame, f"Direction: {direction}", (int(x), int(y) + 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
                
                for line_pts, color in [(line_points_left1, (0, 0, 255)),
                                (line_points_left2, (255, 0, 0)),
                                (line_points_right1, (0, 255, 0)),
                                (line_points_right2, (0, 255, 255))]:
                    cv2.line(frame, line_pts[0], line_pts[1], color, 2)

                #Dibujar linea
                cv2.polylines(frame, [np.array(line_points, dtype=np.int32)], isClosed=True, color=(255, 255, 255), thickness=1)

                #Dibujo tracks
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(frame, [points], isClosed=False, color=(255, 255, 255), thickness=1)
                cv2.circle(frame, (int(track[-1][0]), int(track[-1][1])),  2, (255, 255, 255), -1)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 5 cars, 65.6ms
Speed: 1.0ms preprocess, 65.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 79.7ms
Speed: 2.0ms preprocess, 79.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 66.1ms
Speed: 1.5ms preprocess, 66.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 62.1ms
Speed: 2.0ms preprocess, 62.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 68.1ms
Speed: 1.5ms preprocess, 68.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 58.1ms
Speed: 3.0ms preprocess, 58.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 61.6ms
Speed: 2.0ms preprocess, 61.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 60.1ms
Speed: 2.0ms preprocess, 60.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x